In [9]:
!pip install opencv-python numpy pytesseract


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Motion Analysis:** *Quantify the average motion using Optical Flow.*

In [3]:
import cv2
import numpy as np

def analyze_motion(video_path: str) -> float:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return 0.0

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Cannot read first frame.")
        return 0.0

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    motion_magnitudes = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate optical flow between prev_gray and gray
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, gray, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )

        magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        motion_magnitudes.append(np.mean(magnitude))

        prev_gray = gray

    cap.release()

    avg_motion = float(np.mean(motion_magnitudes)) if motion_magnitudes else 0.0
    return avg_motion

if __name__ == "__main__":
    video_path = "test.mp4"
    avg_motion = analyze_motion("/content/drive/MyDrive/Test_Video/test_video.mp4")
    print(f"Average motion intensity: {avg_motion:.2f}")


Average motion intensity: 0.03
